# Практическое занятие 4. Оценка качества поиска

Давайте реализуем в коде несколько метрик, о которых говорилось на лекции: MAP, NDCG и pFound.

In [ ]:
import math

Для расчёта каждой метрики напишем отдельную функцию. Все функции будут принимать список чисел &mdash; оценки релевантности документов в порядке выдачи &mdash; а также число $k$ &mdash; глубину выдачи, для которой будет рассчитываться метрика.

Напишем сначала вспомогательную функцию для проверки того, что переданные нам оценки находятся в диапазоне $[0, 1]$ и что размер выдачи не меньше $k$. Это пригодится нам в расчёте всех трёх метрик.

In [ ]:
def validate_serp(serp: list[float], k: int):
	assert len(serp) >= k > 0
	assert all(0 <= rel <= 1 for rel in serp)

## 1. Average precision

Average precision рассчитывается для бинарных оценок релевантности, и это надо не забыть проверить. Формула для расчета: $AP@k = {1\over k}\sum\limits_{i=1}^{k} P@i \times Rel(D_i)$

In [ ]:
"""
Дополните/измените код функции ниже, чтобы метрика рассчитывалась верно.
"""
def ap(serp: list[int], k: int) -> float:
    validate_serp(serp, k)
    assert all(rel in [0, 1] for rel in serp)
    sum_ = 0
    for i in range(k):
        if serp[i] == 1:
            sum_ += 1
    return sum_ / k

## 2. DCG

Формула для DCG: $DCG_k = \sum\limits_{i=1}^{k} {{rel_i}\over{\log_2(i+1)}}$. Здесь оценки релевантности уже могут быть любыми (в диапазоне $[0, 1]$).

In [ ]:
def DCG(serp: list[float], k: int) -> float:
    validate_serp(serp, k)
    sum_ = 0
    for i in range(k):
        pass  # ваш код
    return sum_

### NDCG

Вспомним, что NDCG &mdash; это DCG, нормализованный на идеальный DCG для данной коллекции (IDCG). Таким образом, раз считать просто DCG мы уже умеем, то надо научиться считать сначала IDCG, а уже из него рассчитывать NDCG. Формула такова: $IDCG_k = \sum\limits_{i=1}^{|REL_k|}{2^{rel_i} - 1\over{\log_2(i+1)}}$, где $REL_k$ — все релевантные документы, отсортированные по убыванию релевантности.

In [ ]:
def NDCG(serp: list[float], k: int) -> float:
    def IDCG(serp: list[float], k: int) -> float:
        return 0  # ваш код вместо 0

    return DCG(serp, k) / IDCG(serp, k)

## 3. pFound

Идея этой метрики, как и предыдущих, в том, что пользователь просматривает выдачу сверху вниз, но:
- прекращает просмотр с вероятностью, равной релевантности очередного документа,
- либо может прекратить просмотр просто так с вероятностью $pBreak$.

Формулы:

$pFound_k = \sum\limits_{i=1}^{k} pLook_i \times pRel_i$

$pLook_i = pLook_{i-1} \times (1 - pRel_{i-1}) \times (1 - pBreak)$

In [ ]:
def pFound(serp: list[float], k: int, pBreak: float = 0.15) -> float:
    validate_serp(serp, k)
    sum_ = 0
    pLook = [None] * k
    pLook[0] = 1.0  # верхний документ точно будет просмотрен

    for i in range(k):
        # ваш код здесь
        sum_ += pLook[i] * serp[i]
    return sum_

## Тесты

In [ ]:
def check(actual: float, expected: float):
    assert round(actual, 3) == expected, f"Expected value: {expected:.3f}, actual value: {actual:.3f}"

In [ ]:
check(ap([1, 0, 0, 1], 4), 0.375)
check(ap([1, 1, 0, 0], 4), 0.5)

In [ ]:
check(DCG([0.4, 0, 0.2, 0.2, 0], 5), 0.586)
check(NDCG([0.4, 0, 0.2, 0.2, 0], 5), 0.936)

In [ ]:
check(pFound([0.4, 0.1, 0, 0, 0.1], 3), 0.451)
check(pFound([0.4, 0.1, 0.1, 0, 0], 5), 0.49)